In [1]:
abap_method = """Class: cl_edoc_map_ubl
Method Name: map_invoice1
Method Implementation:METHOD map_invoice1.

  DATA:
    ls_target TYPE edo_ubl_invoice_request.

* Set source object
  ms_source = is_source.

* Set document category
  mv_doc_categ = 'INV'.                                     "2854416

* Initial mapping
  init_mapping( ).

* Fill Standard Business DocumentH eader
  ls_target-standard_business_document-standard_business_document_hea = fill_sbdh( ). "2805246, 2854416

* In case of resubmit fill the flag
  ls_target-standard_business_document-resubmit = mv_resubmit.

* Fill Invoice
  ls_target-standard_business_document-invoice = fill_invoice( ).

  es_target = ls_target-standard_business_document.

* Save business partners
  determine_all_bupas( ).

* Call cloud badi
  call_cloud_badi_invoice( CHANGING cs_target = es_target ).

* Add error messages to AIF
  raise_exception( ).

ENDMETHOD."""

In [2]:
abap_method2 = """Class: cl_edoc_map_ubl
Method Name: map_appresp1
Method Implementation:METHOD map_appresp1. "Added with SAP Note 3328617
  CLEAR: es_target.
*  Call INIT_MAPPING
*  Fill SBDH and ApplicationResponse from XML of incoming invoice
*    •  Use GET_VALUE_FROM_XML of CL_EDOC_UTIL to read data from XML
*    •  Note: The sender of the BLR is the receiver of the incoming document and vice versa
*  Fill ResponseCode, StatusReasonCode and StatusReason from mapping source structure
  DATA: ls_target TYPE edo_ubl_application_response_1.
  FIELD-SYMBOLS <ls_document_response> TYPE edo_ubl_document_response.
  FIELD-SYMBOLS <ls_document_reference> TYPE edo_ubl_document_reference2.
  FIELD-SYMBOLS <ls_response_status> TYPE edo_ubl_status.
  FIELD-SYMBOLS <ls_status_reason> TYPE edo_ubl_status_reason.
  TYPES: BEGIN OF ty_s_status_reason_code,
           status_reason_code TYPE edoc_eu_response_code,
         END OF ty_s_status_reason_code.
  DATA ls_status_reason_code TYPE ty_s_status_reason_code.
  DATA lt_op_status_action TYPE SORTED TABLE OF ty_s_status_reason_code WITH NON-UNIQUE KEY status_reason_code.
  DATA lt_op_status_reason TYPE SORTED TABLE OF ty_s_status_reason_code WITH NON-UNIQUE KEY status_reason_code.
  DATA: lo_cx_uuid_error TYPE REF TO cx_uuid_error,
        lv_error_msg     TYPE bapi_msg.

  DATA: ls_fix_value TYPE ty_fix_value,
        lv_docid     TYPE string,
        lv_doctype   TYPE string.

  ms_source = is_source.

  init_mapping( ).

  LOOP AT mt_fix_value_list INTO ls_fix_value WHERE fixvaluename = 'BLR_OP_STATUS_ACTION'.
    ls_status_reason_code-status_reason_code = ls_fix_value-fieldvalue.
    INSERT ls_status_reason_code INTO TABLE lt_op_status_action.
  ENDLOOP.

  LOOP AT mt_fix_value_list INTO ls_fix_value WHERE fixvaluename = 'BLR_OP_STATUS_REASON'.
    ls_status_reason_code-status_reason_code = ls_fix_value-fieldvalue.
    INSERT ls_status_reason_code INTO TABLE lt_op_status_reason.
  ENDLOOP.

  get_blr_document_reference(
    EXPORTING
      iv_invoice_in_source_file = ms_source-file
    IMPORTING
      ev_document_reference = lv_docid
      ev_document_type_code = lv_doctype ).

* Fill Standard Business Document Header
  ls_target-standard_business_document-standard_business_document_hea = fill_sbdh_blr( iv_invoice_in_source_file = ms_source-file ).
  ls_target-standard_business_document-application_response-customization_id-base-base-content = mv_customization_id.
  ls_target-standard_business_document-application_response-profile_id-base-base-content = mv_process_id.
  TRY.
      ls_target-standard_business_document-application_response-id-base-base-content = cl_system_uuid=>create_uuid_c36_static( ).
    CATCH cx_uuid_error INTO lo_cx_uuid_error.
      lv_error_msg = lo_cx_uuid_error->get_text( ).
      RAISE EXCEPTION TYPE cx_edocument
        EXPORTING
          mv_message = lv_error_msg       "passthrough the error message as previous exception is not always considered in output
          previous   = lo_cx_uuid_error.
  ENDTRY.

  ls_target-standard_business_document-application_response-issue_date-base-content = sy-datum.
  ls_target-standard_business_document-application_response-issue_time-base-content = sy-uzeit.

* Fill ApplicationResponse
  DATA lv_sender_prefix TYPE string.
  DATA lv_sender TYPE string.
  DATA lv_receiver_prefix TYPE string.
  DATA lv_receiver TYPE string.

  SPLIT ls_target-standard_business_document-standard_business_document_hea-sender[ 1 ]-identifier-content AT ':' INTO lv_sender_prefix lv_sender.
  SPLIT ls_target-standard_business_document-standard_business_document_hea-receiver[ 1 ]-identifier-content AT ':' INTO lv_receiver_prefix lv_receiver.
  ls_target-standard_business_document-application_response-sender_party-endpoint_id-base-base-scheme_id = lv_sender_prefix.
  ls_target-standard_business_document-application_response-sender_party-endpoint_id-base-base-content = lv_sender.
  ls_target-standard_business_document-application_response-receiver_party-endpoint_id-base-base-scheme_id = lv_receiver_prefix.
  ls_target-standard_business_document-application_response-receiver_party-endpoint_id-base-base-content = lv_receiver.

  ls_target-standard_business_document-application_response-sender_party-party_legal_entity = fill_blr_sndr_prty_legal_entit( iv_invoice_in_source_file = ms_source-file ).
  ls_target-standard_business_document-application_response-receiver_party-party_legal_entity = fill_blr_recv_prty_legal_entit( iv_invoice_in_source_file = ms_source-file ).
* Document Response
  APPEND INITIAL LINE TO ls_target-standard_business_document-application_response-document_response ASSIGNING <ls_document_response>.
  <ls_document_response>-response-response_code-base-base-content = ms_source-blr-response_code.
  IF ms_source-blr-response_code = cl_edoc_map_ubl=>gc_s_blr_response_code-rejected.
    APPEND INITIAL LINE TO <ls_document_response>-response-status ASSIGNING <ls_response_status>.
    <ls_response_status>-status_reason_code-base-base-content = ms_source-blr-rejection_reason_code.
    READ TABLE lt_op_status_reason TRANSPORTING NO FIELDS WITH KEY status_reason_code = ms_source-blr-rejection_reason_code.
    IF sy-subrc = 0.
      <ls_response_status>-status_reason_code-base-base-list_id = 'OPStatusReason' ##no_text.
    ELSE.
      <ls_response_status>-status_reason_code-base-base-list_id = 'OPStatusAction' ##no_text.
    ENDIF.

    APPEND INITIAL LINE TO <ls_response_status>-status_reason ASSIGNING <ls_status_reason>.
    <ls_status_reason>-base-base-content = ms_source-blr-rejection_reason_text.
    IF <ls_status_reason>-base-base-content IS INITIAL.
      <ls_status_reason>-base-base-content = ms_source-blr-rejection_reason_code.
    ENDIF.
  ENDIF.
  APPEND INITIAL LINE TO <ls_document_response>-document_reference ASSIGNING <ls_document_reference>.
  <ls_document_reference>-id-base-base-content = lv_docid.
  <ls_document_reference>-document_type_code-base-base-content = lv_doctype.
  <ls_document_reference>-issue_date-base-content = ls_target-standard_business_document-application_response-issue_date-base-content.

  es_target = ls_target-standard_business_document.
ENDMETHOD."""

In [3]:
from react_agent.src.util.code_summarizer import CodeSummarizer

print(CodeSummarizer.summarize_code(abap_method))
print(CodeSummarizer.summarize_code(abap_method2))

The `map_invoice1` method in the `cl_edoc_map_ubl` class is designed to map and transform invoice data from a source structure into a target structure (`edo_ubl_invoice_request`), while handling document categorization, resubmission flags, business partner determination, and invoking cloud-based enhancements, with error handling integrated through AIF.
The `map_appresp1` method in the `cl_edoc_map_ubl` class is designed to map and populate an ApplicationResponse structure from an incoming invoice XML, utilizing various helper functions to extract and transform data, and handling document references, response codes, and status information for electronic document processing. It ensures that the sender and receiver details are correctly swapped and fills the response structure with relevant metadata and status details, including handling potential UUID errors.
